In [0]:
from ingestion import *
from components.config import *
%load_ext autoreload
%autoreload 2

In [0]:
cdp_path='/dbfs/mnt/cdpprod/Customer_Profile_Aggregates/'
yesterday=str(max(os.listdir(cdp_path)))
trip_data = spark.read.parquet('dbfs:/mnt/cdpprod/Customer_Profile_Aggregates/'+yesterday+'/')\
        .withColumn('Site', F.when(F.col('SiteGroup') == 'LAS', 'Vegas').otherwise('Region'))\
            .filter(F.col('Mnth')>='2016-01-01')\
                .filter((F.col('TripRvMgmt_Segment')!='Convention')|F.col('TripRvMgmt_Segment').isNull())
trip_data = trip_data.withColumn("TripID",F.concat(F.col("Guest_ID"),F.lit('_'), F.col("TripStart"),F.lit('_'),F.col("TripEnd")))

CPA = trip_data.where("property_name != 'BetMGM' and tripstart <= '2025-12-31'").select('guest_id','Property_Name','Department','TripStart','TripEnd','TripStartMlifeTier', 'TripGamingDays','TripID')

trip_CPA = CPA.groupBy('guest_id','TripStart','TripEnd', 'TripID').agg(F.count('Department').alias('dept_num'), F.max('Property_Name').alias('property_name'),
                                                                            F.max('TripGamingDays').alias('TripGamingDays'), F.max('TripStartMlifeTier').alias('TripStartMlifeTier'))
trip_borgata = trip_CPA.where(F.col('property_name').contains("Borgata"))

In [0]:
trip_borgata.where('tripstart <= "2025-01-26" and tripend >= "2025-01-26"').display()

In [0]:
yesterday

In [0]:
return_cus, new_cus = inference_driver()
return_cus = return_cus.cache()

In [0]:
mgm_reward = spark.read.parquet('dbfs:/mnt/proddatalake/prod/CFA/CFA_overall_lvl')
temp = get_return_trip_borgata(mgm_reward)

In [0]:
new_cus.groupby('transactiondate').count().display()

In [0]:
return_cus.where('train_guest_id = 213630030').display()

In [0]:
get_inference_sequence(return_cus, FEATURE_NAMES_RETURN, 6)